## Preparare i Dati per Ditto

Ditto richiede un dataset in formato TSV con questa struttura:

```
sentence_1 \t sentence_2 \t label
```

Esempio:

```
1&1 è una società di telecomunicazioni. \t 1&1 AG è un operatore tedesco di telecomunicazioni. \t 1
AMPLUS è una società di energia. \t AMPLUS SOLAR è una società di energia rinnovabile. \t 0
```

📌 Script per convertire `ground_truth.csv` e `matched_companies.csv` in TSV per Ditto:

In [ ]:
import pandas as pd

# 📂 Carichiamo i dati originali e la ground truth
df_gt = pd.read_csv("data/ground_truth.csv")  # Ground-Truth
df_matched = pd.read_csv("matched_companies.csv")  # RecordLinkage
df_original = pd.read_csv("main_outputs/final_mediated_schema.csv")  # Dataset originale

# 🔍 Controlliamo gli indici originali
print("Colonne in matched_companies:", df_matched.columns.tolist())
print("Colonne in final_mediated_schema:", df_original.columns.tolist())

# 🛠 Ricolleghiamo gli indici di matched_companies.csv ai nomi reali delle aziende
df_matched = df_matched.rename(columns={'Unnamed: 0': 'index_1', 'Unnamed: 1': 'index_2'})

# Convertiamo gli indici a numeri interi
df_matched[['index_1', 'index_2']] = df_matched[['index_1', 'index_2']].astype(int)

# Facciamo il merge per recuperare i nomi reali delle aziende
df_matched = df_matched.merge(df_original[['company_name', 'industry', 'employee_count']], 
                              left_on='index_1', right_index=True, how='left')
df_matched = df_matched.merge(df_original[['company_name']], 
                              left_on='index_2', right_index=True, how='left', suffixes=('_1', '_2'))

# 📌 Prepara il dataset per Ditto (Ground-Truth)
gt_pairs = []
for _, row in df_gt.iterrows():
    sentence_1 = f"{row['company_a']} è una società con sede in {row['source_a']}."
    sentence_2 = f"{row['company_b']} è una società con sede in {row['source_b']}."
    gt_pairs.append(f"{sentence_1}\t{sentence_2}\t1")

# 📌 Prepara il dataset per Ditto (RecordLinkage Matches)
rl_pairs = []
for _, row in df_matched.iterrows():
    sentence_1 = f"{row['company_name_1']} è un'azienda in {row['industry']}."
    sentence_2 = f"{row['company_name_2']} è un'azienda con {row['employee_count']} dipendenti."
    rl_pairs.append(f"{sentence_1}\t{sentence_2}\t0")

# 📝 Salviamo i file in formato TSV con UTF-8 per evitare errori di encoding
with open("ditto/data/ditto_train.tsv", "w", encoding="utf-8") as f:
    f.write("\n".join(gt_pairs))

with open("ditto/data/ditto_test.tsv", "w", encoding="utf-8") as f:
    f.write("\n".join(rl_pairs))

print("✅ File TSV preparati per Ditto!")

## Confrontare i Risultati tra RecordLinkage e Ditto

Ora confrontiamo le prestazioni di RecordLinkage vs Ditto.

📌 Script per calcolare precision, recall e F1-score:

In [ ]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# 📂 Carichiamo i risultati
df_rl = pd.read_csv("matched_companies.csv")  # RecordLinkage
df_ditto = pd.read_csv("data/ditto_results.tsv", sep="\t", header=None, names=["sentence_1", "sentence_2", "label"])
df_gt = pd.read_csv("data/ground_truth.csv")  # Ground-Truth

# 🔄 Convertiamo in set per il confronto
set_rl = set(df_rl["company_name"])
set_ditto = set(df_ditto[df_ditto["label"] == 1]["sentence_1"])
set_gt = set(df_gt["company_a"])

# 📊 Calcoliamo Precision, Recall e F1-score
precision_rl, recall_rl, f1_rl, _ = precision_recall_fscore_support(
    [1 if x in set_gt else 0 for x in set_rl],
    [1 for _ in set_rl], average='binary')

precision_ditto, recall_ditto, f1_ditto, _ = precision_recall_fscore_support(
    [1 if x in set_gt else 0 for x in set_ditto],
    [1 for _ in set_ditto], average='binary')

# 📌 Creiamo il report
comparison = pd.DataFrame({
    "Metodo": ["RecordLinkage", "Ditto"],
    "Precision": [precision_rl, precision_ditto],
    "Recall": [recall_rl, recall_ditto],
    "F1-Score": [f1_rl, f1_ditto]
})

# 📝 Salviamo il confronto
comparison.to_csv("comparison_report.csv", index=False)
print("✅ Confronto completato! Salvato in 'comparison_report.csv'.")
